In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul 31 17:34:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:35:00.0 Off |                    0 |
| N/A   56C    P0              29W /  72W |   7033MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%%capture
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer
!pip install accelerate -U

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) 

from huggingface_hub import notebook_login
import os
import huggingface_hub

from huggingface_hub import HfApi, HfFolder

import os
import huggingface_hub

from huggingface_hub import HfApi, HfFolder
token=os.environ.get("HF_TOKEN")
#huggingface_hub.login(token=token,add_to_git_credential=True, new_session=False, write_permission=True)

api=HfApi(token=token)
folder = HfFolder()
folder.save_token(token)



# huggingface_hub.login(token=os.environ.get("HF_TOKEN"),add_to_git_credential=True, new_session=False, write_permission=True)

In [6]:
%%capture
!apt install git-lfs

TORGO

In [7]:
# import pandas as pd
# import os
# import librosa

# def data_generator(data_dir):
#     metadata_path = os.path.join(data_dir, 'metadata.csv')
#     metadata = pd.read_csv(metadata_path)

#     for _, row in metadata.iterrows():
#         file_path = os.path.join(data_dir, row['file_name'])
#         transcription = row['transcription']
#          # Load the audio file with librosa
#         audio_data, sampling_rate = librosa.load(file_path, sr=16_000)  # Ensure the sampling rate is 16 kHz

#         # Structure the audio data and sampling rate in a dictionary
#         audio_dict = {"array": audio_data, "sampling_rate": sampling_rate}

#         yield {"audio": audio_dict, "path": file_path, "transcription": transcription}


# from datasets import Dataset, Features, Audio, Value

# data_dir = '/content/drive/MyDrive/TORGO-big'  # Adjust this path as needed

# features = Features({
#     "path": Value("string"),
#     "audio": Audio(sampling_rate=16_000),
#     "transcription": Value("string")
# })

# dataset = Dataset.from_generator(
#     generator=data_generator,
#     features=features,
#     gen_kwargs={"data_dir": data_dir}
# )


# # def load_audio(batch):
# #     audio_data, _ = librosa.load(batch['path'], sr=16000)
# #     batch["audio"] = audio_data
# #     return batch


# # dataset = dataset.map(load_audio)

# dataset = dataset.train_test_split(test_size=0.2)


UASPEECH

In [8]:


# import pandas as pd
# import os
# import librosa
# from datasets import Dataset

# def load_parquet_directory(directory_path, use_narrow_data=False):
#     all_dfs = []
#     for file_name in os.listdir(directory_path):
#         if file_name.endswith('.parquet'):
#             file_path = os.path.join(directory_path, file_name)
#             df = pd.read_parquet(file_path)
#             all_dfs.append(df)
#             if use_narrow_data:
#               break

#     return pd.concat(all_dfs, ignore_index=True)

# def convert_dataframe_to_dataset(dataframe):
#     return Dataset.from_pandas(dataframe)


# # def load_audio(batch):
# #     audio_data, _ = librosa.load(batch['path'], sr=16000)
# #     batch["audio"] = audio_data
# #     return batch


# # dataset = dataset.map(load_audio)
# directory_path = '/content/drive/MyDrive/uaspeechall/data' # Update this path
# df = load_parquet_directory(directory_path, use_narrow_data=True)
# dataset = convert_dataframe_to_dataset(df)




In [9]:
import pyarrow.parquet as pq
from datasets import Dataset, concatenate_datasets
import glob
import pyarrow as pa

def chunk_generator(table, chunk_size=10000):
    num_rows = table.num_rows
    for i in range(0, num_rows, chunk_size):
        yield table.slice(i, min(chunk_size, num_rows - i)).to_batches()[0]

def process_parquet_files(directory_path, chunk_size=10000):
    parquet_files = glob.glob(f'{directory_path}/*.parquet')
    for file in parquet_files:
        print(f"Processing file: {file}")
        table = pq.read_table(file, memory_map=True)
        for chunk in chunk_generator(table, chunk_size):
            yield Dataset(pa.Table.from_batches([chunk]))

# Usage
directory_path = '/teamspace/uploads/uaspeechall/data'
all_datasets = []

for chunk_dataset in process_parquet_files(directory_path):
    # Process each chunk here
    # For example, you can apply your transformations:
    # transformed_chunk = chunk_dataset.map(your_transform_function)
    all_datasets.append(chunk_dataset)
# Combine all chunks into a single dataset
dataset = concatenate_datasets(all_datasets)

Processing file: /teamspace/uploads/uaspeechall/data/test-00000-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00001-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00002-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00003-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00004-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00005-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00006-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00000-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00001-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00002-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00003-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00004-of-00018.parquet
Processing file: /teams

In [10]:
dataset

Dataset({
    features: ['audio', 'speaker_id', 'transcription'],
    num_rows: 2500
})

In [11]:
!pip install torchvision torchaudio pydub

In [14]:
print(dataset[0]["audio"].keys())

dict_keys(['path', 'array', 'sampling_rate'])


In [15]:
# import io
# import numpy as np
# from pydub import AudioSegment

# def convert_audio_format(batch):
#     # Extract the audio data from the dictionary
#     audio_bytes = batch["audio"]["bytes"]  # Replace "data" with the correct key

#     # Convert binary data to audio array
#     audio_data = AudioSegment.from_file(io.BytesIO(audio_bytes), format="wav")
#     # Convert to a NumPy array and normalize
#     audio_array = np.array(audio_data.get_array_of_samples()).astype(np.float32) / 2**15

#     # Update the audio dictionary
#     batch["audio"] = {"array": audio_array, "sampling_rate": 16000}
#     return batch

# # Apply the conversion function to the dataset
# dataset = dataset.map(convert_audio_format)

In [16]:
dataset = dataset.train_test_split(test_size=0.2)

In [17]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = dataset["train"]
common_voice_test = dataset["test"]

In [18]:
common_voice_train[0]

{'audio': {'path': 'M14_B1_UW97_M3.wav',
  'array': array([-1.52587891e-04, -2.13623047e-04, -1.52587891e-04, ...,
          1.22070312e-04, -9.15527344e-05,  0.00000000e+00]),
  'sampling_rate': 16000},
 'speaker_id': 'M14_B1_UW97_M3',
 'transcription': 'duck'}

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [19]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

from IPython.display import Audio
import random

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    for _ in range(num_examples):
        index = random.randint(0, len(dataset)-1)
        example = dataset[index]

        print("Transcription:", example["transcription"])
        # Handle audio data: Displaying as an Audio widget if it's in a binary format
        if isinstance(example["audio"], bytes):
            display(Audio(data=example["audio"], rate=16000))  # Adjust the rate as per your dataset
        else:
            print("Audio: Not in binary format")

show_random_elements(common_voice_train, num_examples=10)


Transcription: bogies
Audio: Not in binary format
Transcription: apprehend
Audio: Not in binary format
Transcription: long
Audio: Not in binary format
Transcription: make
Audio: Not in binary format
Transcription: look
Audio: Not in binary format
Transcription: powwow
Audio: Not in binary format
Transcription: to
Audio: Not in binary format
Transcription: to
Audio: Not in binary format
Transcription: Uniform
Audio: Not in binary format
Transcription: not
Audio: Not in binary format


In [20]:
show_random_elements(common_voice_train, num_examples=10)

Transcription: Three
Audio: Not in binary format
Transcription: avenues
Audio: Not in binary format
Transcription: Upward
Audio: Not in binary format
Transcription: some
Audio: Not in binary format
Transcription: into
Audio: Not in binary format
Transcription: look
Audio: Not in binary format
Transcription: to
Audio: Not in binary format
Transcription: Left
Audio: Not in binary format
Transcription: thou
Audio: Not in binary format
Transcription: Delete
Audio: Not in binary format


In [21]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
    return batch

In [22]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [23]:
show_random_elements(common_voice_train)

Transcription: apprehend
Audio: Not in binary format
Transcription: tab
Audio: Not in binary format
Transcription: said
Audio: Not in binary format
Transcription: supreme
Audio: Not in binary format
Transcription: the
Audio: Not in binary format
Transcription: what
Audio: Not in binary format
Transcription: wherewithal
Audio: Not in binary format
Transcription: use
Audio: Not in binary format
Transcription: chauffeur
Audio: Not in binary format
Transcription: have
Audio: Not in binary format


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

Before finalizing the pre-processing, it is always advantageous to consult a native speaker of the target language to see whether the text can be further simplified.
For this blog post, [Merve](https://twitter.com/mervenoyann) was kind enough to take a quick look and noted that "hatted" characters - like `â` - aren't really used anymore in Turkish and can be replaced by their "un-hatted" equivalent, *e.g.* `a`.

This means that we should replace a sentence like `"yargı sistemi hâlâ sağlıksız"` to `"yargı sistemi hala sağlıksız"`.

Let's write another short mapping function to further simplify the text labels. Remember - the simler the text labels, the easier it is for the model to learn to predict those labels.



In CTC, it is common to classify speech chunks into letters, so we will do the same here.
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars.
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [24]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [25]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [26]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [27]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [28]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [29]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [30]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [31]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [32]:
repo_name = "wav2vec2-large-xls-r-300m-dysarthria-big-dataset"

In [34]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/jmaczan/wav2vec2-large-xls-r-300m-dysarthria-big-dataset/commit/81e47391189846541176f5371c39c836294b9def', commit_message='Upload tokenizer', commit_description='', oid='81e47391189846541176f5371c39c836294b9def', pr_url=None, pr_revision=None, pr_num=None)

In [35]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [36]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [37]:
common_voice_train[0]["audio"]

{'path': 'M14_B1_UW97_M3.wav',
 'array': array([-1.52587891e-04, -2.13623047e-04, -1.52587891e-04, ...,
         1.22070312e-04, -9.15527344e-05,  0.00000000e+00]),
 'sampling_rate': 16000}

In [38]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["transcription"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=False, rate=16000)

bachelor


In [39]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["transcription"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"])#.shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: not
Input array shape: [-6.10351562e-05 -1.52587891e-04 -1.52587891e-04 ... -3.05175781e-05
 -3.05175781e-05  0.00000000e+00]
Sampling rate: 16000


In [40]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [41]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### Training


In [42]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [43]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [44]:
wer_metric = load_metric("wer")

/tmp/ipykernel_2504/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [45]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [47]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "jmaczan/wav2vec2-large-xls-r-300m-dysarthria",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [48]:
model.freeze_feature_extractor()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2177: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [51]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=3e-4,
  warmup_steps=500,
  push_to_hub=True,
  save_total_limit=5,
  load_best_model_at_end=True,
  metric_for_best_model="wer"
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [52]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

### Training

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [53]:
trainer.train()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
200,3.196500,1.770863,0.880000
400,1.613500,1.134699,0.764000
600,1.176900,0.934844,0.722000
800,0.815500,0.532968,0.576000
1000,0.596100,0.392654,0.504000
1200,0.443600,0.317147,0.488000
1400,0.358100,0.287668,0.512000
1600,0.293100,0.159001,0.354000
1800,0.219000,0.136982,0.326000
2000,0.191200,0.136156,0.262000


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: U

TrainOutput(global_step=3750, training_loss=0.515586855951945, metrics={'train_runtime': 2845.7505, 'train_samples_per_second': 21.084, 'train_steps_per_second': 1.318, 'total_flos': 5.775265151020725e+18, 'train_loss': 0.515586855951945, 'epoch': 30.0})

In [54]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/jmaczan/wav2vec2-large-xls-r-300m-dysarthria-big-dataset/commit/38f3515a9d2114143471760d0015b51173a330d4', commit_message='End of training', commit_description='', oid='38f3515a9d2114143471760d0015b51173a330d4', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
```

### Evaluation

In [55]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:55: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: Can't load tokenizer for 'wav2vec2-large-xls-r-300m-dysarthria-big-dataset'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'wav2vec2-large-xls-r-300m-dysarthria-big-dataset' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
df = load_parquet_directory(directory_path, use_narrow_data=True)
dataset = convert_dataframe_to_dataset(df)

common_voice_test_transcription = dataset.train_test_split(test_size=0.2)["test"]

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["transcription"].lower())

Prediction:
watches

Reference:
no


### Optional 
Further training phase on next parquet file

In [ ]:
# import pandas as pd
# import os
# import librosa
# from datasets import Dataset

# global_parquet_file_index = 1

# def load_parquet_directory_by_file_index(directory_path, parquet_file_index=1):
#     all_dfs = []
#     current_parquet_file_index = 0
#     for file_name in os.listdir(directory_path):
#         if file_name.endswith('.parquet'):
#             file_path = os.path.join(directory_path, file_name)
#             df = pd.read_parquet(file_path)
#             all_dfs.append(df)
#             if current_parquet_file_index == parquet_file_index:
#               break
#             current_parquet_file_index = current_parquet_file_index + 1

#     return pd.concat(all_dfs, ignore_index=True)

# def convert_dataframe_to_dataset(dataframe):
#     return Dataset.from_pandas(dataframe)


# # def load_audio(batch):
# #     audio_data, _ = librosa.load(batch['path'], sr=16000)
# #     batch["audio"] = audio_data
# #     return batch


# # dataset = dataset.map(load_audio)
# directory_path = '/content/drive/MyDrive/uaspeechall/data'
# df = load_parquet_directory_by_file_index(directory_path, parquet_file_index=True)
# dataset = convert_dataframe_to_dataset(df)




In [ ]:
# import io
# import numpy as np
# from pydub import AudioSegment

# def convert_audio_format(batch):
#     # Extract the audio data from the dictionary
#     audio_bytes = batch["audio"]["bytes"]  # Replace "data" with the correct key

#     # Convert binary data to audio array
#     audio_data = AudioSegment.from_file(io.BytesIO(audio_bytes), format="wav")
#     # Convert to a NumPy array and normalize
#     audio_array = np.array(audio_data.get_array_of_samples()).astype(np.float32) / 2**15

#     # Update the audio dictionary
#     batch["audio"] = {"array": audio_array, "sampling_rate": 16000}
#     return batch

# # Apply the conversion function to the dataset
# dataset = dataset.map(convert_audio_format)

# from datasets import load_dataset, load_metric, Audio

# dataset = dataset.train_test_split(test_size=0.2)

# common_voice_train = dataset["train"]
# common_voice_test = dataset["test"]

# common_voice_train = common_voice_train.map(remove_special_characters)
# common_voice_test = common_voice_test.map(remove_special_characters)

# vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
# vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

# vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

# vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

# vocab_dict["|"] = vocab_dict[" "]
# del vocab_dict[" "]

# vocab_dict["[UNK]"] = len(vocab_dict)
# vocab_dict["[PAD]"] = len(vocab_dict)
# len(vocab_dict)

# import json
# with open('vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

# from transformers import Wav2Vec2CTCTokenizer
# tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

# tokenizer.push_to_hub(repo_name)

# from transformers import Wav2Vec2FeatureExtractor

# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

# from transformers import Wav2Vec2Processor

# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
# common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
# common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)
# data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# wer_metric = load_metric("wer")

# model = Wav2Vec2ForCTC.from_pretrained(
#     "jmaczan/wav2vec2-large-xls-r-300m-dysarthria-big-dataset",
#     attention_dropout=0.0,
#     hidden_dropout=0.0,
#     feat_proj_dropout=0.0,
#     mask_time_prob=0.05,
#     layerdrop=0.0,
#     ctc_loss_reduction="mean",
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),
# )


In [ ]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#   output_dir=repo_name,
#   group_by_length=True,
#   per_device_train_batch_size=8,
#   gradient_accumulation_steps=2,
#   evaluation_strategy="steps",
#   num_train_epochs=30,
#   gradient_checkpointing=True,
#   fp16=True,
#   save_steps=400,
#   eval_steps=400,
#   logging_steps=400,
#   learning_rate=3e-4,
#   warmup_steps=500,
#   save_total_limit=2,
#   push_to_hub=True,
# )

In [ ]:
# from transformers import Trainer

# trainer = Trainer(
#     model=model,
#     data_collator=data_collator,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=common_voice_train,
#     eval_dataset=common_voice_test,
#     tokenizer=processor.feature_extractor,
# )

In [ ]:
# trainer.train()
# trainer.push_to_hub()
